In [5]:
# !pip install networkx

In [6]:
import pandas as pd
import json
import networkx as nx

In [7]:
def build_graph_from_csv(xlsx_path):
    """
    Chuyển dữ liệu dạng bảng (CSV) thành NetworkX Graph với quan hệ:
      - (Author) -> (Title)  : "wrote"
      - (Nickname) -> (Title): "commented_on"
      - (Reply Nick) -> (Nickname): "reply_to"
    """
    # Đọc CSV
    df = pd.read_excel(xlsx_path)

    # Tạo đồ thị (dùng Graph vô hướng hay DiGraph có hướng tuỳ mục đích)
    G = nx.DiGraph()

    for idx, row in df.iterrows():
        # ---- Lấy các cột chính ----
        title = row['Title']
        link = row['Link']
        author = row['Author']
        author_link = row['Author Link']
        category = row['Category']
        comments_count = row['Comments Count']
        date_post = row['Date']
        detailed_title = row['Detailed Title']
        author_pos = row['Author\'s Position']
        content = row['Content']
        
        # ----------------------
        # 1. Node: Bài viết (title)
        # ----------------------
        # ID node là title (chuỗi). Nếu sợ trùng, có thể ghép title+link
        if not G.has_node(title):
            G.add_node(
                title,
                node_type='article',
                link=link,
                category=category,
                comments_count=comments_count,
                date=date_post,
                detailed_title=detailed_title,
                content=content
            )

        # ----------------------
        # 2. Node: Tác giả (author)
        # ----------------------
        # Có thể ghép thêm author_link nếu trùng tên
        author_id = author.strip()
        if not G.has_node(author_id):
            G.add_node(
                author_id,
                node_type='author',
                author_position=author_pos,
                author_link=author_link
            )
        
        # 2.1. Cạnh Author -> Bài viết
        if not G.has_edge(author_id, title):
            G.add_edge(author_id, title, relation='wrote')

        # ----------------------
        # 3. Xử lý cột Comments_Post (JSON)
        # ----------------------
        raw_comments = row['Comments_Post']
        try:
            comments_list = json.loads(raw_comments)  # parse chuỗi JSON
        except:
            comments_list = []

        for comment_data in comments_list:
            nickname = comment_data.get('Nickname', '').strip()
            comment_txt = comment_data.get('Comment', '')
            likes_str = comment_data.get('Likes', '0')
            reply_nicks = comment_data.get('Reply Nicknames', [])
            reply_comments = comment_data.get('Reply Comments', [])

            # 3.1. Tạo node cho Nickname
            if nickname and not G.has_node(nickname):
                G.add_node(
                    nickname,
                    node_type='user'
                )
            
            # 3.2. Cạnh Nickname -> Title (commented_on)
            #     Lưu số likes nếu cần
            if nickname and not G.has_edge(nickname, title):
                likes_val = int(likes_str) if likes_str.isdigit() else 0
                G.add_edge(
                    nickname,
                    title,
                    relation='commented_on',
                    likes=likes_val,
                    comment_text=comment_txt
                )
            
            # 3.3. Quan hệ reply
            #     reply_nicks & reply_comments có thể khác độ dài => cẩn thận
            for rep_nick, rep_comment in zip(reply_nicks, reply_comments):
                rep_nick_id = rep_nick.strip()
                # Tạo node nếu chưa có
                if rep_nick_id and not G.has_node(rep_nick_id):
                    G.add_node(rep_nick_id, node_type='user')

                # Ở đây, giả sử rep_nick là người Reply tới nickname
                # => (rep_nick) -> (nickname), relation = 'reply_to'
                if rep_nick_id and not G.has_edge(rep_nick_id, nickname):
                    G.add_edge(
                        rep_nick_id,
                        nickname,
                        relation='reply_to',
                        reply_text=rep_comment
                    )

    return G


In [9]:

# 5. Chạy thử hàm build_graph_from_csv
# ---------------------------
xlsx_path = "D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles.xlsx"  # Đường dẫn tới file CSV thực tế của bạn
G = build_graph_from_csv(xlsx_path)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Ví dụ in ra một vài node
sample_nodes = list(G.nodes(data=True))[:5]
for n in sample_nodes:
    print("Node ID:", n[0], "- Attributes:", n[1])

# Ví dụ in ra một vài cạnh
sample_edges = list(G.edges(data=True))[:5]
for e in sample_edges:
    print("Edge:", e)

Number of nodes: 1247
Number of edges: 1000
Node ID: Vé Metro miễn phí - Attributes: {'node_type': 'article', 'link': 'https://vnexpress.net/ve-metro-mien-phi-4832075.html', 'category': 'Kinh doanh & quản trị', 'comments_count': 20.0, 'date': 'Thứ năm, 26/12/2024, 00:00 (GMT+7)', 'detailed_title': 'Vé Metro miễn phí', 'content': 'Một trong những đề tài chúng tôi nói đến gần đây là Lễ hội đường phố (Carnival) tại Nice (Pháp) vào tháng hai. Đây là lễ hội thuộc hàng lớn nhất thế giới và có rất nhiều hoạt động miễn phí cho những người đang sinh sống ở đây. Vé miễn phí nhưng để vào lễ hội, người tham dự phải đặt và nhận vé từ trước cũng như xuất trình vé vào cổng.\nCác em thắc mắc, sao đã miễn phí rồi còn "vẽ vời" ra chuyện "đặt và nhận vé", sao không xả cửa tự do? Tôi giải thích cặn kẽ cho sinh viên của mình về tác dụng và ý nghĩa của việc phát hành vé 0 đồng. Câu chuyện này có thể minh họa rõ hơn bằng thực tế vừa diễn ra trong ngày đầu Metro Bến Thành - Suối Tiên được đưa vào vận hành thư

In [10]:
# 1. Số nút và số cạnh
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Số nút (nodes):", num_nodes)
print("Số cạnh (edges):", num_edges)

# 2. Liệt kê bậc (degree) của từng nút (đối với Graph vô hướng)
#    Nếu G là DiGraph (có hướng), ta có in_degree và out_degree
if isinstance(G, nx.Graph):
    degrees = dict(G.degree())
    print("Degree của từng nút:", degrees)
elif isinstance(G, nx.DiGraph):
    in_degs = dict(G.in_degree())
    out_degs = dict(G.out_degree())
    print("In-degree:", in_degs)
    print("Out-degree:", out_degs)

# 3. Bậc trung bình (average degree)
if isinstance(G, nx.Graph):
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    print("Bậc trung bình (average degree):", avg_degree)

# 4. Mật độ (density) - đo lường tỉ lệ cạnh / tối đa có thể
density = nx.density(G)
print("Mật độ (density):", density)


Số nút (nodes): 1247
Số cạnh (edges): 1000
Degree của từng nút: {'Vé Metro miễn phí': 1, 'Võ Nhật Vinh': 63, 'Visa quá cảnh ngặt nghèo': 1, 'Đặng Thái Hoàng': 1, 'Sơn vàng xe đón trẻ': 1, 'Trình Phương Quân': 26, "Đối phó 'cò đất' bỏ cọc": 1, 'Bùi Mẫn': 14, 'Sau Mr Pips này còn Pips khác': 1, 'Ngô Trọng Thanh': 5, 'Cách đáp trả của Xuân Son': 1, 'Hà Đức Trí': 17, 'Khóc người có công': 1, 'Bùi Võ': 10, 'Vì sao sợ sinh con?': 1, 'Tô Thức': 25, 'Kỹ năng đọc viết: nhà giàu cũng khóc': 1, 'Lâm Vũ': 4, 'Côn đồ đốt quán hát': 1, 'Nguyễn Minh Hoàng': 6, 'A Lủ chuyển đổi số': 1, 'Phạm Hùng Phong': 2, 'Chưa giàu đã già': 1, 'Cẩm Hà': 13, "Người già 'bốn không'": 1, 'Lê Tuyết': 22, "'Thối não' vì sống ảo": 1, "Bữa trưa với 'bố già AI'": 1, 'Nguyễn Xuân Phong': 2, 'Sập bẫy Mr Pips': 1, 'Đức Nguyễn': 7, 'Lười sinh, ngại nuôi con': 1, 'Ai sẽ ở lại, ai phải ra đi?': 1, 'Lê Viết Thái': 1, 'Bắt đầu từ việc nhỏ': 1, 'Đỗ Thành Long': 2, 'Tái sinh dòng Tô Lịch': 1, 'Quan Thế Dân': 43, 'An toàn điện hạt nh

In [11]:
deg_centrality = nx.degree_centrality(G)
print("Degree centrality:", deg_centrality)


Degree centrality: {'Vé Metro miễn phí': 0.0008025682182985554, 'Võ Nhật Vinh': 0.05056179775280899, 'Visa quá cảnh ngặt nghèo': 0.0008025682182985554, 'Đặng Thái Hoàng': 0.0008025682182985554, 'Sơn vàng xe đón trẻ': 0.0008025682182985554, 'Trình Phương Quân': 0.02086677367576244, "Đối phó 'cò đất' bỏ cọc": 0.0008025682182985554, 'Bùi Mẫn': 0.011235955056179775, 'Sau Mr Pips này còn Pips khác': 0.0008025682182985554, 'Ngô Trọng Thanh': 0.0040128410914927765, 'Cách đáp trả của Xuân Son': 0.0008025682182985554, 'Hà Đức Trí': 0.01364365971107544, 'Khóc người có công': 0.0008025682182985554, 'Bùi Võ': 0.008025682182985553, 'Vì sao sợ sinh con?': 0.0008025682182985554, 'Tô Thức': 0.020064205457463884, 'Kỹ năng đọc viết: nhà giàu cũng khóc': 0.0008025682182985554, 'Lâm Vũ': 0.0032102728731942215, 'Côn đồ đốt quán hát': 0.0008025682182985554, 'Nguyễn Minh Hoàng': 0.004815409309791332, 'A Lủ chuyển đổi số': 0.0008025682182985554, 'Phạm Hùng Phong': 0.0016051364365971107, 'Chưa giàu đã già': 0.